# Assignment 5

## Language models


*Joshua Burris*

In this assignment we will explore language models.
To do so, we will encapsulate a language model using a class called `language_model` which implements language models with Laplace add-one smoothing.
An overview of the methods your class should have is as follows:

In [1]:
import string
from collections import defaultdict
from collections import Counter
import math

class language_model:
    def __init__(self, ngram=1) :
        """
        Initialize a language model

        Parameters:
        ngram specifies the type of model:  
        unigram (ngram = 1), bigram (ngram = 2) etc.
        """
        self.ngram = ngram
        

    def train(self, file_name) :
        """
        train a language model

        Parameters:
        file_name is a file that contains the training set for the model
        """
        
        self.text = self.getText(file_name)
        if self.ngram > 1 :
            self.bigram_data = []
            for i in range(len(self.text) - 1) :
                self.bigram_data.append(self.text[i] +' '+ self.text[i+1])
            self.bigram_data = Counter(self.bigram_data)
        if self.ngram > 2 :
            self.trigram_data = []
            for i in range(len(self.text) - self.ngram + 1) :
                gram = self.text[i]
                for j in range(1, self.ngram) :
                    gram += ' '+ self.text[i+j]
                self.trigram_data.append(gram)
            self.trigram_data = Counter(self.trigram_data)
        
        self.frequency_data = Counter(self.text)
        
        self.V = len(self.frequency_data)
        self.total_count = sum(self.frequency_data.values())
        
        pass

    def test(self, file_name) :
        """
        Test a language model on a given text and return the perplexity 
        of a trained model on the text provided as input

        Parameters:
        file_name is a file that contains the test set on which the 
        model needs to be evaluated 
        """
        
        text = self.getText(file_name)
        
        # Calculate sparsity
        zero_entries = 0
        entries = 0
        for i in range(len(text) - self.ngram + 1) :
            gram = text[i]
            for j in range(1, self.ngram) :
                gram += ' '+ text[i+j]
            data = {}
            if self.ngram == 1 :
                data = self.frequency_data
            elif self.ngram == 2 :
                data = self.bigram_data
            elif self.ngram == 3 :
                data = self.trigram_data
            if data.setdefault(gram, 0) == 0 :
                zero_entries += 1
            entries += 1
        self.sparsity = zero_entries / entries
        
        return self.perplexity(text)
    
    def perplexity(self, text) :
        return math.pow(2, self.entropy(text))
    
    def entropy(self, text) :
        e = 0
        for i in range(self.ngram - 1, len(text)) :
            context = text[i - self.ngram + 1 : i] # This is the previous word/words for the bigram/trigram.
            e += -math.log(self.probability(text[i], context), 2)     
        return e / (len(text) - (self.ngram - 1))
        
    '''Word is a single word. Context is a list of words.'''
    def probability(self, word, context) :
        if self.ngram == 1 :
            return (self.count([word]) + 1) / (self.total_count + self.V)
        else :
            return (self.count(context + [word]) + 1) / (self.count(context) + self.V)
    
    def count(self, context) :
        length = len(context)
        context = ' '.join(context)
        if length == 1 :
            return self.frequency_data.setdefault(context, 0)
        elif length == 2 :
            return self.bigram_data.setdefault(context, 0)
        elif length == 3 :
            return self.trigram_data.setdefault(context, 0)
        return 0
    
    def getText(self, filename) :
        
        text = open(filename, 'r').read()
        
        # Convert the text to lower case.
        text = text.lower()

        # Convert question marks (?), colons (:) and exclamation marks (!) to periods.
        # Dashes should be converted to spaces.
        text = text.translate({63: 46, 58: 46, 33: 46, 45: 32})

        # Remove all punctuation marks other than the period (commas, semicolons,
        # underscores and quotes).
        trans = str.maketrans('', '', string.punctuation.replace('.', ''))
        text = text.translate(trans)

        # Also replace whitespace with a single space.
        text = ' '.join(text.split())

        # Parse the text into sentences, adding beginning of sentence and end of sentence tokens.
        sentences = text.split('.')
        text = []
        for i in range(len(sentences)) :
            sentences[i] = sentences[i].strip()
            if sentences[i] is not '' :
                text += ['<s>'] + sentences[i].split() + ['</s>']
        
        return text


### Your tasks:

Complete the above class for training and evaluating
language models using unigrams, bigrams, and trigrams.
Before using a text for training or testing:
* Convert the text to lower case.
* Convert question marks (?), colons (:) and exclamation marks (!) to periods.
* Remove all punctuation marks other than the period (commas, semicolons, underscores and quotes); dashes should be converted to spaces.
* Parse the text into sentences, adding beginning of sentence and end of sentence tokens.

For performing the transformations of the text use Python's string method `str.translate`, which also uses a conversion table that is created by the string method `str.maketrans`.
Finally for storing your language model, use a Python dictionary.  You may find it useful to use Python's `defaultdict`, which is part of the `collections` module.



Using the code you have written, demonstrate how you would use language models to distinguish between the writings of Jane Austen and some of her contemporaries.  We have provided the text for the following books for you to use in ASCII format:
* Pride and Prejudice by Jane Austen
* Persuasion by Jane Austen
* Sense and Sensibility by Jane Austen
* Jane Eyre by Charlotte Bronte
* Alice in Wonderland by Lewis Carroll

These can be downloaded [here](https://www.cs.colostate.edu/~cs440/fall19/assignments/texts.tar.gz).
Original versions of the text are from the [Project Gutenberg](http://www.gutenberg.org/) website. 

In your analysis, compare the ability of unigram, bigram and trigram models for this task.  Explain your observations.


Finally, analyze the models you have constructed for Pride and Prejudice.  How sparse are the bigram and trigram models?  I.e., what fraction of the total number of entries in the model before smoothing are zero?


In [2]:

def get_results() :
    book_list = ['pride_and_prejudice.txt', 'persuasion.txt', 'sense_and_sensibility.txt', 'jane_eyre.txt', 'alice_in_wonderland.txt']
    for i in range(3) :
        print('Gram:', i+1)
        model = language_model(i+1)
        for train_book in book_list :
            model.train(train_book)
            for test_book in book_list :
                print('Train:', train_book, 'Test:', test_book, 'Perplexity:', model.test(test_book))
                if train_book == 'pride_and_prejudice.txt' :
                    print('Sparsity:', model.sparsity)
        print()
    pass

def test_data() :
    gram = 1
    model = language_model(gram)
    model.train('alice_in_wonderland.txt')
    print('Perplexity:', model.test('alice_in_wonderland.txt'))
    print('Vocabulary size:', model.V)
    print('Frequency data (top 10):')
    for k, v in model.frequency_data.most_common(10) :
        print('\t', k + ':', v)
    if gram > 1 :
        print('Bigram data (top 10):')
        for k, v in model.bigram_data.most_common(10) :
            print('\t', k + ':', v)
    if gram > 2 :
        print('Trigram data (top 10):')
        for k, v in model.trigram_data.most_common(10) :
            print('\t', k + ':', v)
    pass

get_results()
#test_data()


Gram: 1
Train: pride_and_prejudice.txt Test: pride_and_prejudice.txt Perplexity: 445.5110872106086
Sparsity: 0.0
Train: pride_and_prejudice.txt Test: persuasion.txt Perplexity: 590.0455193284915
Sparsity: 0.05362720706699839
Train: pride_and_prejudice.txt Test: sense_and_sensibility.txt Perplexity: 571.2597150265282
Sparsity: 0.05007125672341665
Train: pride_and_prejudice.txt Test: jane_eyre.txt Perplexity: 694.1180654023972
Sparsity: 0.10168234476802043
Train: pride_and_prejudice.txt Test: alice_in_wonderland.txt Perplexity: 694.5026331277033
Sparsity: 0.10397111913357401
Train: persuasion.txt Test: pride_and_prejudice.txt Perplexity: 552.7067598222569
Train: persuasion.txt Test: persuasion.txt Perplexity: 456.92171164182287
Train: persuasion.txt Test: sense_and_sensibility.txt Perplexity: 580.5883834898078
Train: persuasion.txt Test: jane_eyre.txt Perplexity: 685.6877970666901
Train: persuasion.txt Test: alice_in_wonderland.txt Perplexity: 656.8630946446768
Train: sense_and_sensibili

## Results:

### Unigram:
| Training set ▼ \| Test set ► | Pride and Prejudice | Persuasion | Sense and Sensibility | Jane Eyre | Alice in Wonderland |
|-|-|-|-|-|-|
| Pride and Prejudice | 446 | 590 | 571 | 694 | 695 |
| Persuasion | 552 | 457 | 581 | 686 | 657 |
| Sense and Sensibility | 555 | 600 | 459 | 697 | 690 |
| Jane Eyre | 682 | 731 | 714 | 545 | 625 |
| Alice in Wonderland | 580 | 586 | 593 | 525 | 279 |

### Bigram:
| Training set ▼ \| Test set ► | Pride and Prejudice | Persuasion | Sense and Sensibility | Jane Eyre | Alice in Wonderland |
|-|-|-|-|-|-|
| Pride and Prejudice | 655 | 1122 | 1100 | 1251 | 1269 |
| Persuasion | 1173 | 735 | 1265 | 1356 | 1309 |
| Sense and Sensibility | 1053 | 1150 | 708 | 1268 | 1265 |
| Jane Eyre | 1944 | 2079 | 2089 | 1106 | 1763 |
| Alice in Wonderland | 803 | 809 | 832 | 737 | 320 |

### Trigram:
| Training set ▼ \| Test set ► | Pride and Prejudice | Persuasion | Sense and Sensibility | Jane Eyre | Alice in Wonderland |
|-|-|-|-|-|-|
| Pride and Prejudice | 2022 | 3900 | 3907 | 4009 | 4231 |
| Persuasion | 3780 | 2007 | 3956 | 3947 | 4080 |
| Sense and Sensibility | 3814 | 3985 | 2120 | 4078 | 4251 |
| Jane Eyre | 7471 | 7738 | 7836 | 3654 | 7297 |
| Alice in Wonderland | 1350 | 1352 | 1365 | 1312 | 604 |

### Pride and Prejudice sparsity comparison:
| Ngram ▼ \| Test set ► | Pride and Prejudice | Persuasion | Sense and Sensibility | Jane Eyre | Alice in Wonderland |
|-|-|-|-|-|-|
| Unigram | 0.000% | 5.363% | 5.007% | 10.168% | 10.397% |
| Bigram | 0.000% | 37.558% | 35.408% | 45.931% | 44.932% |
| Trigram | 0.000% | 74.188% | 73.043% | 78.589% | 78.489% |

## Discussion:
An ngram language model is used to determine how similar two pieces of text are to each other. It's clear that unigrams have the lower perplexity/better model over bigrams and trigrams. The perplexity for unigrams was lower in every scenario. Notably, when comparing the same texts to each other they almost always had the lowest perplexity compared to other novels. I would also say based on the data, Jane Austen's novels were more like her own than her contemporaries. There was a clear lower perplexity when comparing Jane Austen's books to themselves, and a slightly higher perplexity otherwise. 

When comparing the sparsity of Pride and Prejudice to other novels using bigrams and trigrams, bigrams had notably less sparsity. Sparsity is the fraction of the number of entries in the training model over the total number of entries in the testing model. This is expected because it reinforces our conclusion of comparing perplexity. A lower sparsity yields a lower perplexity, meaning the texts are more similar. I would say the bigram models weren't too sparse as the percentage was only around ~40%, but the trigram models were very sparse at around ~75%. The sparsity was also notably higher when comparing Jane Austen to her contemporaries, ~5% higher in each case.

## Submission

Answer the questions in the cells reserved for that purpose.
Submit your report as a Jupyter notebook via Canvas. Running the notebook should generate all the results in your notebook.  Leave the output of your notebook intact so we don't have to run it to see the results.

## Grading

Grading will be based on the following criteria:

* Your code solves the specified problem.
* You performed an analysis that addresses the problem.
* Overall readability and organization of the notebook.
* Effort in making interesting observations where required.
* Conciseness. Points may be taken off if the notebook is overly long.